In [59]:
# requirements
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

In [2]:
# import modules
from scipy import ndimage
import skimage
import numpy as np
import random
import cv2
import copy
import os

In [3]:
# define data augmentation parameters
augmentation_parameters = {
    "grayscale": False,
    "flip": True,
    "rotate": True,
    "shift": True,
    "noise": True,
    "blur": True,
    "rotation_values": {
        "random_angle": True,
        "specific_angle": None
    },
    "shifting_values": {
        "min_shift": 1,
        "max_shift": 25
    },
    "noise_values": {
        "mean": 0,
        "var": 0.01
    },
    "blur_values": {
        "sigma": 1
    }
}

In [4]:
# augmentation operations

# img grayscale
def img_grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# img flip
def img_flip(img):
    flip = {
        0: np.flip,
        1: np.flipud,
        2: np.fliplr
    }

    operation = random.randint(0, 2)
    flipped_img = flip[operation](img)
    return flipped_img

# img rotation
def img_rotate(img):
    random_angle = augmentation_parameters["rotation_values"]["random_angle"]
    specific_angle = augmentation_parameters["rotation_values"]["specific_angle"]

    if random_angle is False and specific_angle is None:
        rotation = {
            0: cv2.ROTATE_90_CLOCKWISE,
            1: cv2.ROTATE_90_COUNTERCLOCKWISE,
            2: cv2.ROTATE_180
        }
        operation = random.randint(0, 2)
        rotated_img = cv2.rotate(img, rotation[operation])
    elif random_angle is True:
        angle = random.randint(0, 360)
        rotated_img = ndimage.rotate(img, angle)
    elif specific_angle is not None:
        angle = int(specific_angle)
        rotated_img = ndimage.rotate(img, angle)
    else:
        print("No rotation could be done to the image, please check the parameters")
        return img

    return rotated_img

# img shifting
def img_shift(img):
    min_shift = augmentation_parameters["shifting_values"]["min_shift"]
    max_shift = augmentation_parameters["shifting_values"]["max_shift"]

    # RGB img
    if len(img.shape) == 3:
        shift = [random.uniform(min_shift, max_shift), random.uniform(min_shift, max_shift), 0]
        shifted_img = ndimage.shift(img, shift)
    # Grayscale img
    else:
        shift = [random.uniform(min_shift, max_shift), random.uniform(min_shift, max_shift)]
        shifted_img = ndimage.shift(img, shift)

    return shifted_img

# adding noise
def img_noise(img):
    mean = augmentation_parameters["noise_values"]["mean"]
    var = augmentation_parameters["noise_values"]["var"]

    modes = ['gaussian', 'localvar', 'poisson', 'salt', 'pepper', 's&p', 'speckle']
    mode = modes[random.randint(0, len(modes) - 1)]

    if mode == 'gaussian' or mode == 'speckle':
        noisy_img = skimage.util.random_noise(img, mean=mean, var=var)
    else:
        noisy_img = skimage.util.random_noise(img)

    return (255 * noisy_img).astype(np.uint8)

# blurring
def img_blur(img):
    sigma = augmentation_parameters["blur_values"]["sigma"]

    blurring_value = random.uniform(0, sigma)
    blurred_img = ndimage.gaussian_filter(img, sigma=blurring_value)
    return blurred_img

In [5]:
# dataset generation from one image

# generate available operations map
available_ops = []

if augmentation_parameters['grayscale'] is True:
    available_ops.append(img_grayscale)
if augmentation_parameters['flip'] is True:
    available_ops.append(img_flip)
if augmentation_parameters['rotate'] is True:
    available_ops.append(img_rotate)
if augmentation_parameters['shift'] is True:
    available_ops.append(img_shift)
if augmentation_parameters['noise'] is True:
    available_ops.append(img_noise)
if augmentation_parameters['blur'] is True:
    available_ops.append(img_blur)

if len(available_ops) == 0:
    raise ValueError("No operations allowed, please check config file for augmentation ops.")

def generate_dataset(img, dataset_size, nr_op, image_prefix=None, save=False, save_path=None):
    dataset = []
    for i in range(dataset_size):
        augmented_img = copy.copy(img)
        for j in range(nr_op):
            operation = random.randint(0, len(available_ops) - 1)
            augmented_img = available_ops[operation](augmented_img)

        if save is True:
            if image_prefix is not None:
                img_name = save_path + str(image_prefix) + '_augmented_' + str(i) + '.jpg'
            else:
                img_name = save_path + 'augmented_' + str(i) + '.jpg'
            cv2.imwrite(img_name, augmented_img)

        dataset.append(augmented_img)

    return dataset

In [6]:
# read images from folder
def read_images(data_folder, img_format):
    # read the images

    image_data = {}
    for img in os.listdir(data_folder):
        read_img_format = os.path.splitext(img)[1]
        if read_img_format in img_format:
            image_data[img] = cv2.imread(data_folder + img, cv2.IMREAD_COLOR)
    print("Image reading complete.")
    return image_data

In [7]:
# image resizing
def resize_images(image_data, desired_image_size, save=False, save_path=None):

    resized_images = {}
    for img in image_data:
        resized_images[img] = cv2.resize(image_data[img], desired_image_size, interpolation=cv2.INTER_AREA)

    # save resized images
    if save is True:
        if os.path.exists(save_path) is False:
            os.makedirs(save_path, exist_ok=True)

        for img in resized_images:
            path = save_path + img
            cv2.imwrite(path, resized_images[img])
    print("Image resizing complete.")

    return resized_images

In [8]:
# augment a image into a dataset
def augment_dataset(image_data, dataset_size=50, nr_op=5, save=False, save_path=None):
    # augment a dataset
    complete_dataset = {}
    for img in image_data:
        img_name = os.path.splitext(img)[0]
        if save is True:
            temp_save_path = save_path + img_name + '/'
            if save is True and os.path.exists(temp_save_path) is False:
                os.makedirs(temp_save_path, exist_ok=True)

        temp_dataset = generate_dataset(image_data[img], dataset_size, nr_op, image_prefix=img_name, save=save, save_path=temp_save_path)
        complete_dataset[img_name] = temp_dataset

    if save is True:
        print("Data augmentation completed successfully. Results can be found in {0}".format(save_path))
        return complete_dataset
    else:
        print("Data augmentation completed successfully.")
        return complete_dataset

In [9]:
# load a image and generate some data
images_folder = './data/originals/'
images_format = '.jpg'
images_data = read_images(images_folder, images_format)

Image reading complete.


In [10]:
# resize the images to be sure that they are in the correct format
resized_images_save_path = './data/original_resized/'
desired_size = (128, 128)
resized_images = resize_images(images_data, desired_size, True, resized_images_save_path)

Image resizing complete.


In [11]:
# augment images into datasets
# define parameters
dataset_size = 200  # images generated / input image
nr_op = 5  # nr. of operations done per image
augmented_dataset_save_path = './data/augmented_dataset/'
augmented_dataset = augment_dataset(resized_images, dataset_size, nr_op, True, augmented_dataset_save_path)

# do a final resize to be sure the images are the same size
resized_imgs = {}
for img_type in augmented_dataset.keys():
    resized_imgs[img_type] = []
    for img in augmented_dataset[img_type]:
        resized_img = cv2.resize(img, desired_size, interpolation=cv2.INTER_AREA)
        resized_imgs[img_type].append(resized_img)

augmented_dataset = resized_imgs

Data augmentation completed successfully. Results can be found in ./data/augmented_dataset/


In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, InputLayer, Dropout
from keras.utils.np_utils import to_categorical

# create the architecture of a simple model
def simple_classifier(classes_nr, input_shape=(128, 128, 3)):

    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=3, strides=3, activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, strides=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_nr, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [13]:
# in case you have a graphics card installed
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# train the model
# create train data
labels = {}
label_index = 0
for key in augmented_dataset.keys():
    labels[key] = label_index
    label_index += 1

classes_nr = len(list(labels.keys()))

train_data = []
label_data = []

for key, value in augmented_dataset.items():
    label_value = labels[key]
    for img in value:
        train_data.append(img)
        label_data.append(label_value)

train_data = np.asarray(train_data).astype(np.float32)
label_data = np.asarray(label_data).astype(np.float32)
label_data = to_categorical(label_data, classes_nr)

model = simple_classifier(classes_nr)
model.fit(train_data, label_data, batch_size=4, epochs=100, verbose=1, shuffle=True)
model.save('test_model.h5')

Epoch 1/100
250/250 [==============================] - 10s 3ms/step - loss: 30.2416 - accuracy: 0.2031
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 1.6704 - accuracy: 0.2890
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 1.4997 - accuracy: 0.3237
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 1.4681 - accuracy: 0.3712
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 1.3090 - accuracy: 0.4422
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 1.2246 - accuracy: 0.5199
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 1.1623 - accuracy: 0.5220
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 1.0730 - accuracy: 0.5849
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 1.0830 - accuracy: 0.5996
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.8868 - accu

250/250 [==============================] - 1s 3ms/step - loss: 0.1839 - accuracy: 0.9322
Epoch 82/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2602 - accuracy: 0.9018
Epoch 83/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2191 - accuracy: 0.9384
Epoch 84/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1635 - accuracy: 0.9384
Epoch 85/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1948 - accuracy: 0.9246
Epoch 86/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1929 - accuracy: 0.9344
Epoch 87/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2186 - accuracy: 0.9353
Epoch 88/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2407 - accuracy: 0.9163
Epoch 89/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3439 - accuracy: 0.9232
Epoch 90/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2317 - accuracy: 